In [ ]:
from sympy import *
init_printing()
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
x = Symbol('x', positive=True)
y = Function('y')

In [ ]:
dgl = Eq(y(x).diff(x), sqrt(x*y(x)))
dgl

In [ ]:
lsg = dsolve(dgl)
lsg

In [ ]:
phi = lsg.rhs

In [ ]:
w_liste = [Rational(1,50), Rational(1,9)]
f_list = []
for w in w_liste:
    ab_glg = Eq(phi.subs(x,1), w)
    ab_lsg = solve({ab_glg})
    for l in ab_lsg:
        f_ab = phi.subs(l).simplify()
        display(f_ab)
        f_list.append(f_ab)

In [ ]:
fn_list = []
for f_ab in f_list:
    fn = lambdify(x, f_ab)
    fn_list.append(fn)

In [ ]:
xn = np.linspace(0, 1.2)
fig = plt.figure()
ax = fig.add_subplot(111)
for ff in fn_list:
    ax.plot(xn, ff(xn))

5 Verstöße gegen Picard-Lindelöf

In [ ]:
nx = 15
ny = 19 
xq = np.linspace(0, 1.2, nx)
yq = np.linspace(0, .5, ny)
X, Y = np.meshgrid(xq, yq)
vf = np.zeros_like(X)
for j in range(nx):
    for k in range(ny):
        vf[k,j] = dgl.rhs.subs({x: xq[j], y(x): yq[k]})
U = np.ones_like(X)
V = vf
ax.quiver(X, Y, U, V, angles='xy');

In [ ]:
fig

Alle fallenden Äste sind Unsinn!

# Bernoullische Differentialgleichungen

Eine Differentialgleichung der Form
  \begin{equation*}
    y' + g(x) y + h(x) y^\alpha = 0. \qquad (1)
  \end{equation*}
heißt *Bernoullische Differentialgleichung*.

Durch die Substitution $y=w^{1/(1-\alpha)}$ wird sie in eine lineare Differentialgleichung transformiert.  Das geht aber nur dort, wo $w>0$.

In [ ]:
alpha = Rational(1,2)
1/(1-alpha)

In [ ]:
w = Function('w', positive=True)

In [ ]:
ldgl = dgl.subs(y(x), w(x)**2)
ldgl

In [ ]:
tmp = ldgl.doit()
tmp

In [ ]:
tmp_lsg = solve(tmp, w(x).diff(x))
tmp_lsg

In [ ]:
dgl2 = Eq(w(x).diff(x), tmp_lsg[0])
dgl2

Die Anfangsbedingung $y(1)=\frac19$ übersetzt sich zu $w(1)=\frac13$.

In [ ]:
ics = {w(1): Rational(1,3)}
lsg = dsolve(dgl2, ics=ics)
lsg

In [ ]:
phi = lsg.rhs**(1/(1-alpha))
phi

In [ ]:
dgl.subs(y(x), phi).doit()

Wo kommt die falsche Lösung her?

Es gilt $y = w^{\frac1{1-\alpha}}$.  In diesem Beispiel ist $\frac1{1-\alpha} = 2$.

Setzt man also $w(1) = -\frac13$, dann hat man ebenfalls $y(1)=\frac19$.  Das geht aber nichtb gut:

In [ ]:
ics = {w(1): -Rational(1,3)}
lsg2 = dsolve(dgl2, ics=ics)
lsg2

In [ ]:
phi2 = lsg2.rhs**(1/(1-alpha))
phi2

In [ ]:
glg = dgl.subs(y(x), phi2).doit()
glg

In [ ]:
glg.subs(x,1)

# Mayavi

`conda install -c conda-forge mayavi ipywidgets ipyevents`

In [ ]:
from mayavi import mlab
mlab.init_notebook()

In [ ]:
rn = np.linspace(0, 5*np.pi)
theta_n = np.linspace(0, 2*np.pi)
R, Theta = np.meshgrid(rn, theta_n)
X = R*np.cos(Theta)
Y = R*np.sin(Theta)
W = np.cos(R)

In [ ]:
mlab.mesh(X, Y, W)

In [ ]:
mlab.close()
mlab.mesh(X, Y, W, scalars=1/(R+10), colormap='winter')

# Das Möbiusband

In [ ]:
def moebiusband():
    t = Symbol('t')  
    s = Symbol('s')  # Seele bei s=0
    M = Matrix([ 3*cos(t) + s*sin(t/2),
                       3*sin(t),
                       s*cos(t/2)])
    sn = np.linspace(-1, 1)
    tn = np.linspace(0, 2*np.pi)
    S1, T1 = np.meshgrid(sn, tn)
    fn = [lambdify((s,t), M[j]) for j in range(3)]
    Xn = [fn[j](S1, T1) for j in range(3)]
    return M, Xn, S1   

In [ ]:
M, Xn, S1 = moebiusband()

In [ ]:
try:
    mlab.close()
except AttributeError:
    pass
mlab.mesh(Xn[0], Xn[1], Xn[2], scalars=S1)

In [ ]:
def normale(M):
    s = Symbol('s')
    t = Symbol('t')
    seele = M.subs(s, 0)
    Mt = seele.diff(t)
    Ms = M.diff(s)
    display(Mt, Ms)
    N = Mt.cross(Ms)   # Kreuzprodukt
    display(N)
    N_ein = seele + s*N/N.norm()
    sn = np.linspace(0.05, 1)
    tn = np.linspace(0, 2*np.pi)
    S1, T1 = np.meshgrid(sn, tn)
    Nn = [lambdify((s,t), N_ein[j]) for j in range(3)]
    Xn = [Nn[j](S1, T1) for j in range(3)]
    return Xn

In [ ]:
Nn = normale(M)

In [ ]:
mlab.mesh(Nn[0], Nn[1], Nn[2], color=(0, .3, 0)) 

# Klausurdurchführung

https://www.math.uni-duesseldorf.de/~internet/compana22/posts/klausurdurchfuhrung/